In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta

In [ ]:
csv_path_launch_ts = "job_launch_ts.csv"
df_launch = pd.read_csv(csv_path_launch_ts).fillna("")
# deduplication by run_name, take the first one
print(df_launch.shape)
df_launch = df_launch.sort_values(by='run_name', ascending=False).drop_duplicates(subset='run_name', keep='first')
print(df_launch.shape)
df_launch.sample(2)

In [ ]:
csv_path_jobs = "wandb_runs_time-to-train.csv"
df_jobs = pd.read_csv(csv_path_jobs)
print(df_jobs.shape)
# rename total_time to total_job_time
df_jobs = df_jobs.rename(columns={"total_time": "total_job_time"})
df_jobs.sample(2)

In [ ]:
df = pd.merge(df_jobs, df_launch, on="run_name", how="left").fillna("")
print(df.shape)
df.sample(2)

In [ ]:
from dateutil import parser

def get_total_time_from_launch(launch_time_str: str, end_time_str: str) -> float:
    if not launch_time_str or not end_time_str:
        return

    launch_time = parser.parse(launch_time_str)
    end_time = parser.parse(end_time_str)
    diff = end_time - launch_time
    diff_seconds = diff.total_seconds()
    diff_hrs = diff_seconds / 3600.0
    return diff_hrs

df["total_time_from_launch"] = df.apply(lambda row: get_total_time_from_launch(row["launch_time"], row["heartbeat_time"]), axis=1)

In [ ]:
def get_tpu_type(run_name: str) -> str:
    # default value
    tpu_type = "v5lite-256"
    tpu_zone = "us"

    if "v4-256" in run_name or "v4" in run_name:
        tpu_type = "v4-256"
    else:
        tpu_type = "v5lite-256"
    
    if "europe-west4-b" in run_name:
        tpu_zone = "eu"
    else:
        tpu_zone = "us"
    return f"{tpu_type}({tpu_zone})"

df["tpu_type"] = df["run_name"].apply(get_tpu_type)
df["tpu_type"].value_counts()

In [ ]:
"""
I have a dataframe with the following columns:
- launch_time: the time a job is launched
- create_time: the time when a cluster is allocated for the job
- heartbeat_time: the last time a job is active
- tpu_type: the type of the TPU that the job is using/going to use

I want to create a function that, for each row, calculates that
by the time the job is launched, how many clusters of the same tpu_type
are actively running.

Think thought this problem step-by-step, then write the function
"""
def count_active_clusters(df):
    def parse_date(date_str):
        if not date_str:
            return ""
        if isinstance(date_str, datetime):
            return date_str
        return parser.parse(date_str)

    for col in ['launch_time', 'heartbeat_time', 'create_time']:
        df[col] = df[col].apply(parse_date)

    # Initialize a list to store the counts
    cluster_counts_at_launch, cluster_counts_at_running = [], []

    for _, row in df.iterrows():
        tpu_type = row['tpu_type']
        create_time = row["create_time"]

        # Count active clusters of the same tpu_type at launch_time
        # count = sum(
        #     (df["tpu_type"] == row['tpu_type']) &
        #     (df["heartbeat_time"] > create_time) & (df["create_time"] < create_time)
        # )
        count = sum(
            (df["tpu_type"] == row['tpu_type']) &
            # (df["heartbeat_time"] > create_time) & (df["create_time"] < create_time)
            (df["heartbeat_time"] + timedelta(minutes=10) > create_time) & (df["create_time"] < create_time)
        )
        cluster_counts_at_launch.append(count)

        # Count ative clusters between create_time and heartbeat_time
        count = sum(
            (df["tpu_type"] == row['tpu_type']) &
            (df["heartbeat_time"] + timedelta(minutes=10) > create_time) & (df["create_time"] < row["heartbeat_time"])
        )
        cluster_counts_at_running.append(count)

    df['clusters_at_create'] = cluster_counts_at_launch
    df['clusters_at_running'] = cluster_counts_at_running

    return df

# Usage:
# df = pd.read_csv('your_data.csv')  # Load your dataframe
# df_with_counts = count_active_clusters(df)


df = count_active_clusters(df)

In [ ]:
import seaborn as sns


def get_time_to_launch(row):
    if "total_time_from_launch" not in row:
        return ""
    return row["total_time_from_launch"] - row["total_job_time"]

df["time_to_create_cluster"] = df.apply(get_time_to_launch, axis=1)

# Get unique TPU types for color assignment
tpu_types = df['tpu_type'].unique()
# colors = plt.cm.plasma(np.linspace(0, 1, len(tpu_types)))
colors = sns.color_palette("husl", len(tpu_types))


In [ ]:
df.loc[df["tpu_type"] == "v5lite-256(eu)", ["run_name", "runtime", "clusters_at_create", "launch_time", "create_time", "time_to_create_cluster", "heartbeat_time"]].to_csv("temp.csv")

In [ ]:
# Create two subplots side by side
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 5))

# Add a main title
fig.suptitle('Cluster Creation Time vs # Active Clusters at Creation, grouped by TPU Type', fontsize=16)

# Function to plot data on a given axis
def plot_data(ax):
    for tpu_type, color in zip(tpu_types, colors):
        mask = df['tpu_type'] == tpu_type
        ax.scatter(df.loc[mask, 'clusters_at_create'],
                   df.loc[mask, 'time_to_create_cluster'],
                   c=[color],
                   label=tpu_type,
                   alpha=0.7,
                   s=15)
    
    ax.set_xlabel('# Active Clusters at Creation')
    ax.set_ylabel('Time to Create Cluster (hrs)')
    ax.grid(True, linestyle='--', alpha=0.7)
    ax.legend()

# Plot on the first subplot (full range)
plot_data(ax1)
ax1.set_title('Full Range of Y-axis')

# Plot on the second subplot (y-axis from 0 to 2)
plot_data(ax2)
ax2.set_ylim(0, 2)
ax2.set_title('Y-axis (0 - 2)')

# Adjust layout
plt.tight_layout()

# Adjust the layout to make room for the main title
plt.subplots_adjust(top=0.88)

# Show the plot
plt.show()


In [ ]:
df["effective_training"] = df["runtime"] / df["total_job_time"]

def get_time_to_1t(row):
    seq_len = 4096
    target = 1000000000000.0
    train_batch_size = row["train_batch_size"]
    global_step = row["global_step"]
    total_job_time = row["total_job_time"]
    total_steps = target / seq_len / train_batch_size
    time_to_train = total_steps / global_step * total_job_time
    return time_to_train


df["time_to_train_1T"] = df.apply(get_time_to_1t, axis=1)

# filter out jobs whose training_time is less than 1 hr
df_filtered = df[df["training_time"] > 1]
len(df), len(df_filtered)

In [ ]:
# Create a scatter plot
plt.figure(figsize=(8, 5))

# Plot each TPU type with a different color
for tpu_type, color in zip(tpu_types, colors):
    mask = df_filtered['tpu_type'] == tpu_type
    plt.scatter(df_filtered.loc[mask, 'clusters_at_create'],
                df_filtered.loc[mask, 'effective_training'],
                c=[color],
                label=tpu_type,
                alpha=0.7)

# Set labels and title
plt.xlabel('# Active Clusters during Creation')
plt.ylabel('Effective Training Time (% of Total Run Time)')
plt.title('Effective Training Time vs # Active Clusters during Creation, grouped by TPU Type')

# Add legend
plt.legend()

# Show grid
plt.grid(True, linestyle='--', alpha=0.7)

# Show the plot
plt.tight_layout()
plt.show()


In [ ]:
# Create a scatter plot
plt.figure(figsize=(8, 5))

# Plot each TPU type with a different color
for tpu_type, color in zip(tpu_types, colors):
    mask = df_filtered['tpu_type'] == tpu_type
    plt.scatter(df_filtered.loc[mask, 'clusters_at_running'],
                df_filtered.loc[mask, 'effective_training'],
                c=[color],
                label=tpu_type,
                alpha=0.7)

# Set labels and title
plt.xlabel('# Active Clusters during Training')
plt.ylabel('Effective Training Time (%)')
plt.title('Effective Training vs # Active Clusters during Training, grouped by TPU Type')

# Add legend
plt.legend()

# Show grid
plt.grid(True, linestyle='--', alpha=0.7)

# Show the plot
plt.tight_layout()
plt.show()


In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np

# Set the style for better-looking plots
plt.style.use('ggplot')

# Get unique TPU types
tpu_types = df_filtered['tpu_type'].unique()

# Create two subplots side by side
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Function to plot histogram on a given axis
def plot_histogram(ax, x_range=None, bins=20):
    for tpu_type in tpu_types:
        subset = df_filtered[df_filtered['tpu_type'] == tpu_type]
        ax.hist(subset['time_to_train_1T'], bins=bins, range=x_range, alpha=0.7, label=tpu_type)
    
    ax.set_xlabel('Time to Train 1T (hrs)', fontsize=12)
    ax.set_ylabel('Count', fontsize=12)
    ax.tick_params(axis='both', which='major', labelsize=10)
    ax.legend(title='TPU Type', title_fontsize='12', fontsize='10')

# Plot on the first subplot (full range)
plot_histogram(ax1, bins=50)
ax1.set_title('Full Range of X-axis', fontsize=14)

# Plot on the second subplot (x-axis from 0 to 200)
plot_histogram(ax2, x_range=(100, 300))
ax2.set_title('X-axis Range (100 to 300 hrs)', fontsize=14)

# Add a main title
fig.suptitle('Distribution of Time to Train 1T, grouped by TPU Type', fontsize=16)

# Adjust the layout manually
plt.tight_layout()
plt.subplots_adjust(top=0.88)  # Make room for the main title

# Show the plot
plt.show()


In [ ]:
df.to_csv("wandb_runs_with_launch_ts.csv", index=False)